In [37]:
import requests
import time 

import pandas as pd
import numpy as np
from web3 import Web3


In [38]:
ETHERSCAN_API_KEY = '16FCD3FTVWC3KDK17WS5PTWRQX1E2WEYV2'
ETHERSCAN_URL = "https://api.etherscan.io/api"
POOL0_ADDRESS="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640" # USDC / WETH (0.05%) 
POOL0_TXN_FEE = 0.0005
POOL1_ADDRESS="0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8" # USDC / WETH (0.3%)
POOl1_TXN_FEE = 0.003

GWEI_SCALER = 1e9

In [39]:
def fetch_latest_block(url, api_key):
    # Parameters for the API request
    params = {
        "module": "proxy",
        "action": "eth_blockNumber",
        "apikey": api_key
    }
    
    # Make the request
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        #st.error(f"API request failed with status code {response.status_code}")
        (f"API request failed with status code {response.status_code}") 
        return None
    
    # Handle the response
    if response.status_code == 200:
        result = response.json()
        if result.get("status") != "0" and "result" in result:
            latest_block = int(result['result'], 16)  # Convert hex to int
            #print(f"Latest Block Number: {latest_block}")
            return latest_block
        else:
            print(f"Unexpected response or error: {result.get('message', 'Unknown error')}")
            return None
    else:
        print(f"Failed to fetch the latest block. Status code: {response.status_code}")
        return None
    


In [74]:
def fetch_data(url, api_key, pool_address, start_block, end_block):
    params = {
        'module': 'account',
        'action': 'tokentx',
        'address': pool_address,
        'startblock': startblock,
        'endblock': endblock,
        'sort': 'desc',
        'apikey': api_key
    }
    
    response = requests.get(url, params=params)
    if response.status_code != 200:
        #st.error(f"API request failed with status code {response.status_code}")
        raise Exception(f"API request failed with status code {response.status_code}")
    
    data = response.json()
    if data['status'] != '1':
        #st.error(f"API returned an error: {data['result']}")
        raise Exception(f"API returned an error: {data['result']}")
    
    df = pd.DataFrame(data['result'])
    
    expected_columns = ['hash', 'blockNumber', 'timeStamp', 'from', 'to', 'gas', 'gasPrice', 'gasUsed', 'cumulativeGasUsed', 'confirmations', 'tokenSymbol', 'value', 'tokenName']
    
    for col in expected_columns:
        if col not in df.columns:
            raise Exception(f"Expected column '{col}' is missing from the response")
    
    #df.sort_values(by='timeStamp')
    
    consolidated_data = {}

    for index, row in df.iterrows():
        tx_hash = row['hash']
        
        if tx_hash not in consolidated_data:
            consolidated_data[tx_hash] = {
                'blockNumber': int(row['blockNumber']),
                'timeStamp': row['timeStamp'],
                'transactionHash': tx_hash,
                'from': row['from'],
                'to': row['to'],
                'WETH_value': 0,
                'USDC_value': 0,
                'tokenName_WETH': '',
                'tokenName_USDC': '',
                'gas': row['gas'],
                'gasPrice': int(row['gasPrice']),
                'gasUsed': int(row['gasUsed']),
                'cumulativeGasUsed': int(row['cumulativeGasUsed']),
                'confirmations': row['confirmations']
            }
        
        if row['tokenSymbol'] == 'WETH':
            consolidated_data[tx_hash]['WETH_value'] = row['value']
            consolidated_data[tx_hash]['tokenName_WETH'] = row['tokenName']
        elif row['tokenSymbol'] == 'USDC':
            consolidated_data[tx_hash]['USDC_value'] = row['value']
            consolidated_data[tx_hash]['tokenName_USDC'] = row['tokenName']
    df = pd.DataFrame.from_dict(consolidated_data, orient='index').reset_index(drop=True)
    return df.sort_values(by='timeStamp')

In [75]:
# fetch block numbers for the last hour.
# Assumes that there is at least one transaction from both pools
# and more than ten transactions total.
endblock = fetch_latest_block(ETHERSCAN_URL, ETHERSCAN_API_KEY)
startblock = endblock-5 * 60
if endblock: 
    print(f"Start Block: {startblock}")
    print(f"Stop Block: {endblock}")

try: 
    p0 = fetch_data(ETHERSCAN_URL, ETHERSCAN_API_KEY, POOL0_ADDRESS, startblock, endblock)
    p0 = p0.iloc[-10:].reset_index(drop=True)

    endtime = int(p0['timeStamp'].iloc[0])
    beginningtime = int(p0['timeStamp'].iloc[-1])
    print(f"Successfully fetched {p0.shape[0]} swaps in the last {(endtime -beginningtime)/60:.2f} minutes.")
    
    p1 = fetch_data(ETHERSCAN_URL, ETHERSCAN_API_KEY, POOL1_ADDRESS, startblock, endblock)
    p1 = p1.iloc[-10:].reset_index(drop=True)
    endtime = int(p1['timeStamp'].iloc[0])
    beginningtime = int(p1['timeStamp'].iloc[-1])
    print(f"Successfully fetched {p1.shape[0]} swaps in the last {(endtime -beginningtime)/60:.2f} minutes.")
except Exception as e: 
    print(e)

Start Block: 21617102
Stop Block: 21617402
Successfully fetched 10 swaps in the last -0.80 minutes.
Successfully fetched 10 swaps in the last -6.20 minutes.


In [89]:
p0

,blockNumber,timeStamp,transactionHash,from,to,WETH_value,USDC_value,tokenName_WETH,tokenName_USDC,gas,gasPrice,gasUsed,cumulativeGasUsed,confirmations
0,21617397,1736793407,0xd7765d5acb9e933299e5ed00ac06d512fb66780744e6...,0x2b619d5501baeec7947a43671d7d59255d89c710,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,16592589103862770,50000000,Wrapped Ether,USDC,313571,8075149818,222045,6972652,6
1,21617397,1736793407,0x09179d419eac91671c4dc752450059330bb4581d4200...,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,1175922465431949205,3539910785,Wrapped Ether,USDC,236644,8075149818,164776,8267909,6
2,21617397,1736793407,0xcd630b9ff4e630b593df524e086fb3cddc092cb8ef7e...,0x68d3a973e7272eb388022a5c6518d9b2a2e66fbf,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,31627477759071573588,95259234726,Wrapped Ether,USDC,307042,95979990679,153785,697688,6
3,21617398,1736793419,0xe24dadb2edf3956e5bf713e07e3763ae3ee1ad287ae3...,0x1f2f10d1c40777ae1da742455c65828ff36df387,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,18758789871718891520,56452622161,Wrapped Ether,USDC,415505,33916677987,290854,405553,5
4,21617398,1736793419,0xa3940682fdf11bffaecd444646edeca0af8f60038b04...,0xa69babef1ca67a37ffaf7a485dfff3382056e78c,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,23163237087899407693,69781888484,Wrapped Ether,USDC,332218,6160363931,146109,551662,5
5,21617398,1736793419,0xa0759aa517af6e272154c30e3654aba15536b3ddf416...,0xf081470f5c6fbccf48cc4e5b82dd926409dcdd67,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,5579142710325342484,16795583594,Wrapped Ether,USDC,2020815,7257604631,1163785,14032674,5
6,21617399,1736793431,0xf6a1acd641b87cd263496121232301c165282a1a3bf5...,0x68d3a973e7272eb388022a5c6518d9b2a2e66fbf,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,24175580271728850779,72872618712,Wrapped Ether,USDC,247042,69849519063,123835,123835,4
7,21617399,1736793431,0xe4f3b40d2824d347c55ba12e181e248f514112904b9c...,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,797466328801812514,2402277005,Wrapped Ether,USDC,363845,8565040701,260691,2934949,4
8,21617400,1736793443,0xbe3cd65afde66604a4d205ccf690e821ceab2563bcee...,0x1f2f10d1c40777ae1da742455c65828ff36df387,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,14968631969171111936,45080275344,Wrapped Ether,USDC,373331,24043860375,261332,261332,3
9,21617401,1736793455,0x45267d3729f292c83e10eaee55593db9edb519af0773...,0xd4bc53434c5e12cb41381a556c3c47e1a86e80e3,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,34812497465108856832,104762089100,Wrapped Ether,USDC,488358,88367331824,341851,462031,2


In [78]:
p0[['timeStamp','blockNumber','transactionHash']].drop_duplicates().shape

(10, 3)

In [79]:
def decode_block_data(url, swap_tx_data, block_number,transaction_hash,datatype_list,verbose=False):
    """
    
    # Call for a single block
    print(f"Block Number: {block_number}")
    swap_tx_data = pool1_tx_data[0]['data']
    block_number = int(pool1_tx_data[0]['blockNumber'],16)
    block_data = decode_block_data(swap_tx_data,block_number,verbose=True)
    
    """
    w3 = Web3(Web3.HTTPProvider(url))

    amount0, amount1, sqrtPriceX96, liquidity, tick = w3.eth.codec.decode(datatype_list,bytes.fromhex(swap_tx_data[2:]))
    return sqrtPriceX96

# Function to fetch logs from a block
def fetch_logs_for_block(api_key, block_number, pool_address):

    base_url = "https://api.etherscan.io/api"

    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": block_number,
        "toBlock": block_number,
        "address": pool_address,
        "apikey": api_key,
    }
    data = {'result':[]}
    #while not data['result']:
    response = requests.get(base_url, params=params)
    data = response.json()
    #print(".",end='')

    if data.get("status") == "1" and "result" in data:
        return data["result"]
    else:
        raise ValueError(f"ValueError fetching logs for block {block_number}: {data.get('message', 'Unknown error')}")

# Decode the sqrtPriceX96 from the logs
def decode_data(logs,transaction_hashes):
    base_url = "https://api.etherscan.io/api"
    w3 = Web3(Web3.HTTPProvider(base_url))

    # Uniswap V3 Swap event ABI
    # The topic for the Swap event in Uniswap V3 (Swap event signature)
    # https://www.4byte.directory/event-signatures/?bytes_signature=0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67
    swap_event = "0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67"
    swap_event_abi = {
        "anonymous": False,
        "inputs": [
            {"indexed": True, "internalType": "address", "name": "sender", "type": "address"},
            {"indexed": False, "internalType": "int256", "name": "amount0", "type": "int256"},
            {"indexed": False, "internalType": "int256", "name": "amount1", "type": "int256"},
            {"indexed": False, "internalType": "uint160", "name": "sqrtPriceX96", "type": "uint160"},
            {"indexed": False, "internalType": "uint128", "name": "liquidity", "type": "uint128"},
            {"indexed": False, "internalType": "int24", "name": "tick", "type": "int24"}
        ],
        "name": "Swap",
        "type": "event"
    }
    # For Decoding Data Field
    datatype_list = [i['type'] for i in swap_event_abi['inputs'] if not i['indexed']]
    raw_data = []
    for log in logs:
        if (swap_event in log["topics"]) and (log["transactionHash"] in transaction_hashes):
            block = int(log['blockNumber'],16)
            transaction_hash=log['transactionHash']
            #print(f"processing {transaction_hash}, {block}")
            data = log['data']
            amount0, amount1, sqrt_price_x96, liquidity, tick = w3.eth.codec.decode(datatype_list,bytes.fromhex(data[2:]))
            raw_data.append((block,amount0, amount1, sqrt_price_x96, liquidity, tick,transaction_hash))
    return raw_data

def fetch_block_data(etherscan_api_key, block_numbers, time_stamps, transaction_hashes, pool_address):
    """
    works for etherscan_requests_v2
    """
    results = {'timeStamp':[], 'blockNumber':[], 'sqrtPriceX96':[], 'transactionHash':[],'tick':[],'amount0':[],'amount1':[],'liquidity':[]}

    for block,timestamp in zip(block_numbers,time_stamps):
        try:
            logs = fetch_logs_for_block(etherscan_api_key, block, pool_address)
            #import pdb
            #pdb.set_trace()
        except Exception as e:
            print(f"Error fetching logs: {e} (ignoring for now)")
        
        try:
            data_in_block = decode_data(logs,transaction_hashes)

            for block,amount0, amount1, sqrt_price_x96, liquidity, tick,transaction_hash in data_in_block:
                results['timeStamp'].append(timestamp)
                results['blockNumber'].append(block)
                results['sqrtPriceX96'].append(sqrt_price_x96)
                results['transactionHash'].append(transaction_hash)
                results['tick'].append(tick)
                results['liquidity'].append(liquidity)
                results['amount0'].append(amount0)
                results['amount1'].append(amount1)
            print(".",end='')

        except Exception as e:
            print(f"Error decoding logs {block}: {e} (ignoring for now)")
        
    #df =
    #df.set_index('transactionHash',inplace=True)
    
    return  pd.DataFrame(results) 



In [80]:
start_time = time.time()
p0_blockData = fetch_block_data(ETHERSCAN_API_KEY, 
                                list(p0['blockNumber'].unique()),
                                list(p0['timeStamp'].unique()),
                                list(p0['transactionHash']), 
                                POOL0_ADDRESS )
p0_new = p0.merge(p0_blockData,on=['timeStamp','blockNumber','transactionHash'],how='left').dropna().reset_index(drop=True)
p0_new.columns = ['blockNumber', 'timeStamp', 'transactionHash', 'p0.from', 'p0.to',
       'p0.WETH_value', 'p0.USDC_value', 'p0.tokenName_WETH', 'p0.tokenName_USDC', 'p0.gas',
       'p0.gasPrice', 'p0.gasUsed', 'p0.cumulativeGasUsed', 'p0.confirmations',
       'p0.sqrtPriceX96', 'p0.tick', 'p0.amount0', 'p0.amount1', 'p0.liquidity']
finish_time = time.time()
p1_blockData = fetch_block_data(ETHERSCAN_API_KEY, list(p1['blockNumber'].unique()), list(p1['timeStamp'].unique()),list(p1['transactionHash']), POOL1_ADDRESS )
p1_new = p1.merge(p1_blockData,on=['timeStamp','blockNumber','transactionHash'],how='left').dropna().reset_index(drop=True)
p1_new.columns = ['blockNumber', 'timeStamp', 'transactionHash', 'p1.from', 'p1.to',
       'p1.WETH_value', 'p1.USDC_value', 'p1.tokenName_WETH', 'p1.tokenName_USDC', 'p1.gas',
       'p1.gasPrice', 'p1.gasUsed', 'p1.cumulativeGasUsed', 'p1.confirmations',
       'p1.sqrtPriceX96', 'p1.tick', 'p1.amount0', 'p1.amount1', 'p1.liquidity']
finish_time = time.time()
print(f"Completed fetching {p0_new.shape[0]+p1_new.shape[0]} blocks in {finish_time-start_time}s")

..............Completed fetching 20 blocks in 4.220552921295166s


In [81]:
print(p0_new.shape, p0.shape)
print(p1_new.shape, p1.shape)

(10, 19) (10, 14)
(10, 19) (10, 14)


In [82]:
p1_new

,blockNumber,timeStamp,transactionHash,p1.from,p1.to,p1.WETH_value,p1.USDC_value,p1.tokenName_WETH,p1.tokenName_USDC,p1.gas,p1.gasPrice,p1.gasUsed,p1.cumulativeGasUsed,p1.confirmations,p1.sqrtPriceX96,p1.tick,p1.amount0,p1.amount1,p1.liquidity
0,21617368,1736793059,0x7044573e3e30245e6f35ce71113c6a3f412540240482...,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,6799948428944492876,20374535185,Wrapped Ether,USDC,158858,19170662295,122199,2532672,35,1449279288073362659543008394278005,196294,20374535185,-6799948428944492876,908083860934724714
1,21617376,1736793155,0x0837fd1126e1187b3a2203df0e951b17c9fce9f0c6e3...,0x68d3a973e7272eb388022a5c6518d9b2a2e66fbf,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,8054311812040933894,24154547514,Wrapped Ether,USDC,247050,28380134057,123523,238220,27,1448576568475936835219571892444171,196285,24154547514,-8054311812040933894,908083860934724714
2,21617377,1736793167,0x423e20152d87bca4a5fc56924569c311e5fa3f3e206c...,0x58f7b1786cd5a7b292ebe0da8c8ce9dd32d8335d,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,48911137906456456,146754067,Wrapped Ether,USDC,399834,8688225229,224694,5016884,26,1448572301095192042001204896351391,196284,146754067,-48911137906456456,908083860934724714
3,21617380,1736793203,0x76652a282da0fec529082bf7859c44e3f5d510c76846...,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,8992456980729704655,26995868852,Wrapped Ether,USDC,157493,34945450543,121149,518938,23,1447787730557468327387745925592021,196274,26995868852,-8992456980729704655,908083860934724714
4,21617381,1736793215,0xa204909a96232fe192df4ca6a693a4ca836bc38e9b22...,0x68d3a973e7272eb388022a5c6518d9b2a2e66fbf,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,4566752616580634230,13720857320,Wrapped Ether,USDC,247050,8762445588,122167,5300487,22,1447389292225906678193074875963466,196268,13720857320,-4566752616580634230,908083860934724714
5,21617384,1736793251,0x6b416513c06eeafa8d5ba10eaef33ea7f52403e60242...,0x5050e08626c499411b5d0e0b5af0e83d3fd82edf,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,7335558010101937231,22055582527,Wrapped Ether,USDC,303568,6897210562,151784,1731885,19,1446749459460464432213056140832104,196259,22055582527,-7335558010101937231,917878519363806848
6,21617392,1736793347,0xc9f96aeb2b7505e88711965fdd631e3e2176207d46f0...,0x68d3a973e7272eb388022a5c6518d9b2a2e66fbf,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,4892196734770050715,14720007986,Wrapped Ether,USDC,247050,10891393846,123544,243153,11,1446327181621102023782722202212928,196253,14720007986,-4892196734770050715,917878519363806848
7,21617397,1736793407,0x47e4c916b70602900efbb87ff2482fdcd6c4754b20e9...,0x68d3a973e7272eb388022a5c6518d9b2a2e66fbf,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,10121353509028798244,30481189956,Wrapped Ether,USDC,247050,43632217337,122464,820152,6,1445453540693941578168375196956964,196241,30481189956,-10121353509028798244,917878519363806848
8,21617397,1736793407,0x7b196b7a89e8996e6264bb68cc219ca4057a53a73b35...,0xc95c6010681003f8fb493194de062b1a43011368,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,178455832032171582,537763236,Wrapped Ether,USDC,358835,8075149818,258446,11399571,6,1445438136991421485553415532173454,196241,537763236,-178455832032171582,917878519363806848
9,21617399,1736793431,0x5bcf463a4d5d70bf515ac8d469c4ac08659c55c39f8d...,0x68d3a973e7272eb388022a5c6518d9b2a2e66fbf,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,5503591930229603988,16590289821,Wrapped Ether,USDC,247050,13552544263,123490,493201,4,1444963085591196269183777240947611,196235,16590289821,-5503591930229603988,917878519363806848


In [83]:
list(p0_new['transactionHash'])

['0xd7765d5acb9e933299e5ed00ac06d512fb66780744e6e786637d8da5993eded5',
 '0x09179d419eac91671c4dc752450059330bb4581d4200a26dec641278a8414a17',
 '0xcd630b9ff4e630b593df524e086fb3cddc092cb8ef7e5fe9b05289ad77f09ee5',
 '0xe24dadb2edf3956e5bf713e07e3763ae3ee1ad287ae302a79937e940ed7cdd90',
 '0xa3940682fdf11bffaecd444646edeca0af8f60038b040fe50eecdcb12f3ce887',
 '0xa0759aa517af6e272154c30e3654aba15536b3ddf4160436c1a0fc1e1e5dd09b',
 '0xf6a1acd641b87cd263496121232301c165282a1a3bf5e7230c3b2d9341e931cf',
 '0xe4f3b40d2824d347c55ba12e181e248f514112904b9c423624e6212f21c4603e',
 '0xbe3cd65afde66604a4d205ccf690e821ceab2563bcee33f459eab83111441f39',
 '0x45267d3729f292c83e10eaee55593db9edb519af0773301eb0fab166fbb5d734']

In [91]:
print(startblock, endblock)
both_pools = pd.merge(p0_new, p1_new,on=['timeStamp','blockNumber','transactionHash'],how='outer')
both_pools = both_pools.ffill().reset_index(drop=True)
display(both_pools.sort_values(by='blockNumber'))

21617102 21617402


,blockNumber,timeStamp,transactionHash,p0.from,p0.to,p0.WETH_value,p0.USDC_value,p0.tokenName_WETH,p0.tokenName_USDC,p0.gas,...,p1.gas,p1.gasPrice,p1.gasUsed,p1.cumulativeGasUsed,p1.confirmations,p1.sqrtPriceX96,p1.tick,p1.amount0,p1.amount1,p1.liquidity
0,21617368,1736793059,0x7044573e3e30245e6f35ce71113c6a3f412540240482...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,158858,1.917066e+10,122199.0,2532672.0,35,1449279288073362659543008394278005,196294.0,2.037454e+10,-6799948428944492876,9.080839e+17
1,21617376,1736793155,0x0837fd1126e1187b3a2203df0e951b17c9fce9f0c6e3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,247050,2.838013e+10,123523.0,238220.0,27,1448576568475936835219571892444171,196285.0,2.415455e+10,-8054311812040933894,9.080839e+17
2,21617377,1736793167,0x423e20152d87bca4a5fc56924569c311e5fa3f3e206c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,399834,8.688225e+09,224694.0,5016884.0,26,1448572301095192042001204896351391,196284.0,1.467541e+08,-48911137906456456,9.080839e+17
3,21617380,1736793203,0x76652a282da0fec529082bf7859c44e3f5d510c76846...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,157493,3.494545e+10,121149.0,518938.0,23,1447787730557468327387745925592021,196274.0,2.699587e+10,-8992456980729704655,9.080839e+17
4,21617381,1736793215,0xa204909a96232fe192df4ca6a693a4ca836bc38e9b22...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,247050,8.762446e+09,122167.0,5300487.0,22,1447389292225906678193074875963466,196268.0,1.372086e+10,-4566752616580634230,9.080839e+17
5,21617384,1736793251,0x6b416513c06eeafa8d5ba10eaef33ea7f52403e60242...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,303568,6.897211e+09,151784.0,1731885.0,19,1446749459460464432213056140832104,196259.0,2.205558e+10,-7335558010101937231,9.178785e+17
6,21617392,1736793347,0xc9f96aeb2b7505e88711965fdd631e3e2176207d46f0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,247050,1.089139e+10,123544.0,243153.0,11,1446327181621102023782722202212928,196253.0,1.472001e+10,-4892196734770050715,9.178785e+17
11,21617397,1736793407,0xd7765d5acb9e933299e5ed00ac06d512fb66780744e6...,0x2b619d5501baeec7947a43671d7d59255d89c710,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,16592589103862770,50000000,Wrapped Ether,USDC,313571,...,358835,8.075150e+09,258446.0,11399571.0,6,1445438136991421485553415532173454,196241.0,5.377632e+08,-178455832032171582,9.178785e+17
10,21617397,1736793407,0xcd630b9ff4e630b593df524e086fb3cddc092cb8ef7e...,0x68d3a973e7272eb388022a5c6518d9b2a2e66fbf,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,31627477759071573588,95259234726,Wrapped Ether,USDC,307042,...,358835,8.075150e+09,258446.0,11399571.0,6,1445438136991421485553415532173454,196241.0,5.377632e+08,-178455832032171582,9.178785e+17
9,21617397,1736793407,0x7b196b7a89e8996e6264bb68cc219ca4057a53a73b35...,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,1175922465431949205,3539910785,Wrapped Ether,USDC,236644,...,358835,8.075150e+09,258446.0,11399571.0,6,1445438136991421485553415532173454,196241.0,5.377632e+08,-178455832032171582,9.178785e+17


In [96]:
newthing = both_pools[['timeStamp','blockNumber','p0.sqrtPriceX96','p1.sqrtPriceX96']]
display(newthing)
#= {
#    'blockNumber':21617402
#}

,timeStamp,blockNumber,p0.sqrtPriceX96,p1.sqrtPriceX96
0,1736793059,21617368,NaN,1449279288073362659543008394278005
1,1736793155,21617376,NaN,1448576568475936835219571892444171
2,1736793167,21617377,NaN,1448572301095192042001204896351391
3,1736793203,21617380,NaN,1447787730557468327387745925592021
4,1736793215,21617381,NaN,1447389292225906678193074875963466
5,1736793251,21617384,NaN,1446749459460464432213056140832104
6,1736793347,21617392,NaN,1446327181621102023782722202212928
7,1736793407,21617397,1443670661654219434285058796758673,1446327181621102023782722202212928
8,1736793407,21617397,1443670661654219434285058796758673,1445453540693941578168375196956964
9,1736793407,21617397,1443670661654219434285058796758673,1445438136991421485553415532173454


In [102]:
newthing.iloc[:10].to_dict(orient='list')

{'timeStamp': ['1736793059',
  '1736793155',
  '1736793167',
  '1736793203',
  '1736793215',
  '1736793251',
  '1736793347',
  '1736793407',
  '1736793407',
  '1736793407'],
 'blockNumber': [21617368,
  21617376,
  21617377,
  21617380,
  21617381,
  21617384,
  21617392,
  21617397,
  21617397,
  21617397],
 'p0.sqrtPriceX96': [nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  1443670661654219434285058796758673,
  1443670661654219434285058796758673,
  1443670661654219434285058796758673],
 'p1.sqrtPriceX96': [1449279288073362659543008394278005,
  1448576568475936835219571892444171,
  1448572301095192042001204896351391,
  1447787730557468327387745925592021,
  1447389292225906678193074875963466,
  1446749459460464432213056140832104,
  1446327181621102023782722202212928,
  1446327181621102023782722202212928,
  1445453540693941578168375196956964,
  1445438136991421485553415532173454]}